In [5]:
from __future__ import print_function
! pip install dropbox
from dropbox import Dropbox

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

import time
from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow.contrib.keras import backend
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import applications

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
from multiprocessing import Pool
import os
# style_map
style_map={}
style_map.update({'painting':'painting.jpg'})
style_map.update({'wave':'wave.png'})
style_map.update({'forest':'forest.jpg'})
style_map.update({'marilyn':'marilyn.jpg'})
style_map.update({'picasso':'picasso.jpg'})
style_map.update({'scream':'scream.jpg'})
style_map.update({'scream':'scream.jpg'})
style_map.update({'van_gouch':'van_gouch.jpg'})
style_map.update({'modern_Art':'modern_Art.jpg'})
style_map.update({'rain-princess':'rain-princess.jpg'})
style_map.update({'night':'night.jpg'})
style_map.update({'gothic':'gothic.jpeg'})
style_map.update({'blocks':'blocks.jpeg'})


fromaddr = "prisma.nyu@gmail.com"
pwd = "shivdivani123"
pool = Pool(processes=os.cpu_count())

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [0]:
def send_email(toaddr, file): 
	msg =MIMEMultipart()

	msg['From'] = fromaddr
	msg['To'] = toaddr
	msg['Subject'] = "NYU Prisma - your image"

	body = "Hi, PFA your prisma image."

	msg.attach(MIMEText(body, 'plain'))

	attachment = open("./"+file, "rb")

	part = MIMEBase('application', 'octet-stream')
	part.set_payload((attachment).read())
	encoders.encode_base64(part)
	part.add_header('Content-Disposition', "attachment; filename= %s" % file)

	msg.attach(part)

	server = smtplib.SMTP('smtp.gmail.com', 587)
	server.starttls()
	server.login(fromaddr, pwd)
	text = msg.as_string()
	server.sendmail(fromaddr, toaddr, text)
	server.quit()      
  

In [0]:
def neural_style_script(content_image, style_image):
  height = 512
  width = 512
  temp_name = content_image
  content_image = Image.open(content_image)
  content_image = content_image.resize((width, height))
  style_image = style_map.get(style_image)
  style_image = Image.open(style_image)
  style_image = style_image.resize((width, height))
  
  content_array = np.asarray(content_image, dtype='float32')[:,:,0:3]
  content_array = np.expand_dims(content_array, axis=0)

  style_array = np.asarray(style_image, dtype='float32')[:,:,0:3]
  style_array = np.expand_dims(style_array, axis=0)
  
  

  content_array[:, :, :, 0] -= 103.939
  content_array[:, :, :, 1] -= 116.779
  content_array[:, :, :, 2] -= 123.68
  content_array = content_array[:, :, :, ::-1]

  style_array[:, :, :, 0] -= 103.939
  style_array[:, :, :, 1] -= 116.779
  style_array[:, :, :, 2] -= 123.68
  style_array = style_array[:, :, :, ::-1]
  
  
  content_image = backend.variable(content_array)
  style_image = backend.variable(style_array)
  combination_image = backend.placeholder((1, height, width, 3))
  
  input_tensor = backend.concatenate([content_image, style_image,combination_image], axis=0)
  model = applications.VGG16(input_tensor=input_tensor, weights='imagenet',include_top=False)
  
  
  content_weight = 0.025
  style_weight = 5.0
  total_variation_weight = 1.0
  loss = backend.variable(0.)
  
  layers = dict([(layer.name, layer.output) for layer in model.layers])
  def content_loss(content, combination):
      return backend.sum(backend.square(combination - content))

  layer_features = layers['block2_conv2']
  content_image_features = layer_features[0, :, :, :]
  combination_features = layer_features[2, :, :, :]

  loss = loss + content_weight * content_loss(content_image_features, combination_features)
  
  def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram
  
  def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = height * width
    return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

  feature_layers = ['block1_conv2', 'block2_conv2',
                    'block3_conv3', 'block4_conv3',
                    'block5_conv3']
  for layer_name in feature_layers:
      layer_features = layers[layer_name]
      style_features = layer_features[1, :, :, :]
      combination_features = layer_features[2, :, :, :]
      sl = style_loss(style_features, combination_features)
      loss += (style_weight / len(feature_layers)) * sl
  
  def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

  loss += total_variation_weight * total_variation_loss(combination_image)
  grads = backend.gradients(loss, combination_image)
  
  outputs = [loss]
  outputs += grads
  f_outputs = backend.function([combination_image], outputs)

  def eval_loss_and_grads(x):
      x = x.reshape((1, height, width, 3))
      outs = f_outputs([x])
      loss_value = outs[0]
      grad_values = outs[1].flatten().astype('float64')
      return loss_value, grad_values

  class Evaluator(object):

      def __init__(self):
          self.loss_value = None
          self.grads_values = None

      def loss(self, x):
          assert self.loss_value is None
          loss_value, grad_values = eval_loss_and_grads(x)
          self.loss_value = loss_value
          self.grad_values = grad_values
          return self.loss_value

      def grads(self, x):
          assert self.loss_value is not None
          grad_values = np.copy(self.grad_values)
          self.loss_value = None
          self.grad_values = None
          return grad_values

  evaluator = Evaluator()
  x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.

  iterations = 5

  for i in range(iterations):
      print('Start of iteration ',i," of ", temp_name)
      start_time = time.time()
      x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                       fprime=evaluator.grads, maxfun=20)
      print('Current loss value:', min_val)
      end_time = time.time()
      print('Iteration %d completed in %ds' % (i, end_time - start_time))
  
  
  x = x.reshape((height, width, 3))
  x = x[:, :, ::-1]
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = np.clip(x, 0, 255).astype('uint8')  
  Image.fromarray(x).save(temp_name)

In [0]:
def workflow(file):
  print("---->",file)
  neural_style_script(file.split("/")[0], file.split("_")[0])
  with open(file.split("/")[0], "rb") as imageFile:
    output = imageFile.read()
  dbx.files_upload(output,'/prisma/output/'+file.split("/")[1]+'/'+file.split("/")[0])
  send_email(file.split("/")[1], file.split("/")[0])

In [0]:
#API Token
dbx = Dropbox('56Avmht1CGkAAAAAAAABS8gYYmjrmVC5HZc6ThJbMT168Y1MrWhY0wjfXUln64i4')

# Iterate over styles folder and download on colab
for entry in dbx.files_list_folder('/prisma/styles/').entries:
  dbx.files_download_to_file("./"+entry.name, '/prisma/styles/'+entry.name)  


while(True):


  # Iterate over output subfolders
  out_sub_folders=[]
  for entry in dbx.files_list_folder('/prisma/output/').entries:
    out_sub_folders.insert(1,entry.name)

  # Iterate over files in each output subfolder and add it to a set 
  out_files = set()
  for folder in out_sub_folders:  
    for entry in dbx.files_list_folder('/prisma/output/'+folder).entries:
      out_files.add(folder+"/"+entry.name)

  in_sub_folders=[]
  to_process_images=[]

  # Iterate over input subfolders 
  for entry in dbx.files_list_folder('/prisma/input/').entries:
    in_sub_folders.insert(1, entry.name)

  # Iterate over each file in input subfolders and check whether in output folder
  # if not add it in to_process_images
  for folder in in_sub_folders:  
    for entry in dbx.files_list_folder('/prisma/input/'+folder).entries:
      if folder+"/"+entry.name not in out_files:
        to_process_images.insert(1,"/prisma/input/"+folder+"/"+entry.name)

  # download from dropbox, files toprocess
  # extract the filename to refer further
  filenames_toproc =[]
  for file in to_process_images:
    tokens= file.split('/')
    filenames_toproc.insert(1,tokens[len(tokens)-1]+"/"+tokens[len(tokens)-2])
    dbx.files_download_to_file("./"+tokens[len(tokens)-1], file)
  print("submitting files for processing " + str(filenames_toproc))
  try:
    for file in filenames_toproc:
    #pool.map(workflow, filenames_toproc)
      workflow(file)
  except Exception as err:
    print("Error occured!", err)


submitting files for processing ['blocks_208fa84e-5b07-4029-bea3-5993e687c38c_image.jpg/ak5146@nyu.edu', 'blocks_118d629b-487c-480b-9141-1fa7b8b9e7d6_ak.jpg/ak5146@nyu.edu', 'blocks_8382a91f-989f-4d7c-b041-b5acb76fd0f1_image.jpg/ak5146@nyu.edu', 'wave_b2eeb66c-26a0-453c-888e-47c0280da565_image.jpg/ak5146@nyu.edu', 'night_bfb2dd34-9d5d-4a36-baea-348e0e350c7c_image.jpg/ak5146@nyu.edu', 'night_05805202-a242-4f07-a13c-93e0f3d26a77_1C1D2181-4DF2-470A-A62A-CF77A83B460B.jpeg/ak5146@nyu.edu', 'painting_fe12c62c-ae31-4f81-b3de-4464fd91adcb_image.jpg/khera.aniruddh@gmail.com', 'painting_73488306-612d-4892-8828-11f469259b84_image.jpg/khera.aniruddh@gmail.com']
----> blocks_208fa84e-5b07-4029-bea3-5993e687c38c_image.jpg/ak5146@nyu.edu
Start of iteration  0  of  blocks_208fa84e-5b07-4029-bea3-5993e687c38c_image.jpg
Current loss value: 723885300000.0
Iteration 0 completed in 19s
Start of iteration  1  of  blocks_208fa84e-5b07-4029-bea3-5993e687c38c_image.jpg
Current loss value: 324896520000.0
Iterat